In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
src_path = './classes'
num_classes = 5

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='training',
    validation_split=0.2,
    seed=123,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    src_path,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    subset='validation',
    validation_split=0.2,
    seed=123
)

In [ ]:
# Data augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),   # Flip images randomly
    tf.keras.layers.RandomRotation(0.1),                     # Rotate images by up to 20%
    tf.keras.layers.RandomZoom(0.1),                         # Zoom in by up to 20%
    tf.keras.layers.RandomContrast(0.1)                      # Change contrast by up to 20%
])

# Augment only the training data
# train_data = train_data.map( lambda x, y: (data_augmentation(x, training=True), y) )

In [20]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data   = val_data.cache().prefetch(buffer_size=AUTOTUNE)

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

# x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)

# reg = tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)
# x = Dense(256, activation='relu', kernel_regularizer=reg)(x)

# x = Dropout(0.5)(x)  # Dropout 50%
# x = Dropout(0.6)(x)


x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)


In [21]:
from sklearn.utils import class_weight
import numpy as np

labels = [y.numpy().argmax(axis=1) for _, y in train_data]

class_labels = np.concatenate(labels, axis=0)

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(class_labels), y=class_labels)

class_weights_dict = dict(enumerate(class_weights))

2024-10-24 14:53:32.537353: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model.summary()

In [24]:
model.fit(train_data, epochs=10, validation_data=val_data)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1788 - loss: 118.5761 - val_accuracy: 0.2643 - val_loss: 44.6118
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 942ms/step - accuracy: 0.3604 - loss: 23.4601 - val_accuracy: 0.2429 - val_loss: 7.3171
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 944ms/step - accuracy: 0.3081 - loss: 5.5607 - val_accuracy: 0.2214 - val_loss: 6.0617
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 955ms/step - accuracy: 0.3708 - loss: 3.3449 - val_accuracy: 0.2571 - val_loss: 4.7850
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 942ms/step - accuracy: 0.4005 - loss: 2.5185 - val_accuracy: 0.2643 - val_loss: 3.2028
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 950ms/step - accuracy: 0.5193 - loss: 1.5264 - val_accuracy: 0.3143 - val_loss: 2.8968
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 955ms/step - accuracy: 0.5243 - loss: 1.6575 - val_accuracy: 0.2786 - val_loss: 4.3439
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 947ms/step - accuracy: 0.4169 - loss: 3.1675 - val_acc